In [1]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from random import sample
import os

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

сделать матрицу

In [2]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [3]:
data_points = pd.read_csv('data/Dewas_points.csv')

In [4]:
data_ndvi = pd.read_csv('data/Dewas_NDVI.csv')

In [5]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [6]:
data['month'] = data['date'].apply(pd.to_datetime).dt.month
data['day'] = data['date'].apply(pd.to_datetime).dt.day

In [7]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,72001,0,2020-10-20,0.186
1,72001,1,2020-10-21,0.184
2,72001,2,2020-10-22,0.183
3,72001,3,2020-10-23,0.182
4,72001,4,2020-10-24,0.181


In [8]:
i = 1
ids_not_full = []
for id_ in data_points['gfid'].values:
    shp = data_ndvi[data_ndvi['gfid'] == id_].shape[0]
    if shp != 203:
        #print(id_, ' ', shp)
        i += 1
        ids_not_full += [id_]
#print(i)

In [9]:
data2 = data[~data['gfid'].isin(ids_not_full)]

In [10]:
data2[data2['gfid']==72001]['ndvi']

0      0.186
1      0.184
2      0.183
3      0.182
4      0.181
       ...  
198    0.123
199    0.121
200    0.120
201    0.119
202    0.118
Name: ndvi, Length: 203, dtype: float64

In [11]:
gfids = data2['gfid'].unique()

In [12]:
from numpy.random import choice

In [13]:
import numpy as np

In [14]:
np.random.seed(42)

In [15]:
np.random.shuffle(gfids)

In [16]:
train_ids = gfids[:300]
test_ids = gfids[300:]

In [17]:
data2.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi,month,day
0,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,0,2020-10-20,0.186,10,20
1,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,1,2020-10-21,0.184,10,21
2,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,2,2020-10-22,0.183,10,22
3,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,3,2020-10-23,0.182,10,23
4,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,4,2020-10-24,0.181,10,24


In [18]:
data2 = data2.dropna()

In [19]:
data2.shape

(76125, 12)

In [20]:
data2.shape

(76125, 12)

In [21]:
data2[data2['gfid']==72001][:150].values.reshape(15,10,12)[0]

array([[72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 0, '2020-10-20', 0.186, 10, 20],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 1, '2020-10-21', 0.184, 10, 21],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 2, '2020-10-22', 0.183, 10, 22],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 3, '2020-10-23', 0.182, 10, 23],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 4, '2020-10-24', 0.181, 10, 24],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 5, '2020-10-25', 0.18, 10, 25],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 6, '2020-10-26', 0.18, 10, 26],
       [72001, 'Madhya Pradesh ', 'Dewas ', 'VijayaganjMandi', 75.96199,
        23.218479, 0, 7, '2020-10

In [22]:
for i in (3, 5, 7, 9):
    data2[f'ndvi_{i}'] = data2['ndvi'].rolling(i).mean()

In [23]:
datasets_train = []
datasets_test = []
y_train = []
y_test = []
for _ in gfids:
    rolling_data = []
    rolling_data.append(data2[data2['gfid']==_]['ndvi'].iloc[4:])
    for i in range(2, 11):
        rolling_data.append(data2[data2['gfid']==_]['ndvi'].rolling(i).mean())
    data_1_rolling = pd.concat(rolling_data, axis=1)
    data_1_rolling = data_1_rolling.dropna()
    data_1_rolling.columns = ['ndvi'] + [f'ndvi_{i}' for i in range(2,11)]
    if _ in train_ids:
        datasets_train.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_train += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]
    else:
        datasets_test.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_test += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]

In [24]:
data_train = torch.stack([torch.Tensor(dataset) for dataset in datasets_train])
data_test = torch.stack([torch.Tensor(dataset) for dataset in datasets_test])

In [25]:
def calc_out(h_in, w_in, ker, pad=0, stride=1):
    h_out = (h_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    w_out = (w_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    return h_out, w_out

In [26]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes  #number of classes
        
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=30, kernel_size=2, stride=2, padding=1),  # (b x 30 x 6 x 6)
            nn.ReLU(),
            nn.Conv2d(30, 64, 2, padding=0, stride=1),  # (b x 64 x 5 x 5)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),  # (b x 64 x 4 x 4)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1, inplace=True),
            nn.Linear(in_features=(64 * 4 * 4), out_features=256),
            nn.ReLU(),
            #nn.Dropout(p=0.5, inplace=True),
            #nn.Linear(in_features=256, out_features=256),
            #nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes),
            nn.Sigmoid()
        )
        #self.init_bias()

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[0].bias, 1)
        nn.init.constant_(self.net[2].bias, 1)
        #nn.init.constant_(self.net[4].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 64 * 4 * 4)  # reduce the dimensions for linear layer input
        return self.classifier(x)

In [27]:
alexnet = AlexNet(num_classes=2)

In [28]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=alexnet.parameters(), lr=0.0001)#1, weight_decay=0.5)

In [29]:
torch.autograd.set_detect_anomaly(True)

X_train_tensors = Variable(torch.Tensor(data2.loc[data2['gfid'].isin(train), features].values.reshape(300, 2, 203)))
X_test_tensors = Variable(torch.Tensor(data2.loc[data2['gfid'].isin(test), features].values.reshape(75, 2, 203)))

y_train_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(train), 'wheat'].values))
y_test_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(test), 'wheat'].values))

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

X = X_train_tensors
y = y_train_tensors

In [30]:
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [31]:
X = data_train
y = y_train_tensors

In [32]:
n_epochs = 100  # or whatever
batch_size = 30  # or whatever

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(X.size()[0])

    for i in range(0, X.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X[indices], y[indices]

        outputs = alexnet.forward(batch_x) #forward pass
        #optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        loss = criterion(outputs, batch_y)
        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.69171
Epoch: 1, loss: 0.69351
Epoch: 2, loss: 0.68580
Epoch: 3, loss: 0.68222
Epoch: 4, loss: 0.68165
Epoch: 5, loss: 0.67500
Epoch: 6, loss: 0.66462
Epoch: 7, loss: 0.63783
Epoch: 8, loss: 0.65397
Epoch: 9, loss: 0.64110
Epoch: 10, loss: 0.67254
Epoch: 11, loss: 0.67359
Epoch: 12, loss: 0.62988
Epoch: 13, loss: 0.64154
Epoch: 14, loss: 0.55613
Epoch: 15, loss: 0.58423
Epoch: 16, loss: 0.61271
Epoch: 17, loss: 0.69231
Epoch: 18, loss: 0.62703
Epoch: 19, loss: 0.68272
Epoch: 20, loss: 0.60988
Epoch: 21, loss: 0.59674
Epoch: 22, loss: 0.56031
Epoch: 23, loss: 0.60889
Epoch: 24, loss: 0.60597
Epoch: 25, loss: 0.63013
Epoch: 26, loss: 0.54985
Epoch: 27, loss: 0.60336
Epoch: 28, loss: 0.62888
Epoch: 29, loss: 0.62314
Epoch: 30, loss: 0.51418
Epoch: 31, loss: 0.56273
Epoch: 32, loss: 0.59610
Epoch: 33, loss: 0.62164
Epoch: 34, loss: 0.55310
Epoch: 35, loss: 0.57632
Epoch: 36, loss: 0.64552
Epoch: 37, loss: 0.62737
Epoch: 38, loss: 0.61804
Epoch: 39, loss: 0.62076
Epoch: 40,

In [33]:
X_test_tensors = data_test

In [34]:
test_predict = alexnet(X_test_tensors)#forward pass 

In [35]:
test_predict

tensor([[9.1478e-01, 9.2831e-02],
        [9.9995e-01, 6.9728e-05],
        [9.8011e-01, 2.2498e-02],
        [8.2170e-01, 1.8874e-01],
        [9.7814e-02, 8.9204e-01],
        [1.0322e-01, 8.8837e-01],
        [9.3063e-01, 7.7998e-02],
        [9.2046e-01, 9.1910e-02],
        [1.5234e-01, 8.3409e-01],
        [9.9150e-01, 1.0091e-02],
        [1.1526e-01, 8.8085e-01],
        [9.9650e-01, 4.2110e-03],
        [1.0944e-01, 8.9311e-01],
        [7.1529e-01, 2.9183e-01],
        [2.9698e-01, 7.0177e-01],
        [8.1107e-01, 2.0550e-01],
        [9.6573e-01, 3.8617e-02],
        [8.8482e-02, 8.9860e-01],
        [9.5565e-01, 5.0043e-02],
        [8.3335e-01, 1.7870e-01],
        [9.9749e-01, 3.1191e-03],
        [1.5558e-01, 8.4841e-01],
        [3.0948e-01, 6.8610e-01],
        [9.0367e-01, 1.0653e-01],
        [1.0385e-01, 8.9307e-01],
        [1.4290e-01, 8.5699e-01],
        [2.7718e-01, 7.2814e-01],
        [5.6065e-01, 4.4025e-01],
        [1.5487e-01, 8.4883e-01],
        [1.094

In [36]:
test_predict.argmax(dim=1)

tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 1])

In [37]:
# 100 эпох, батч 30
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80        40
           1       0.75      0.86      0.80        35

    accuracy                           0.80        75
   macro avg       0.80      0.80      0.80        75
weighted avg       0.81      0.80      0.80        75



In [38]:
train_predict = alexnet(X)#forward pass 

In [39]:
# 100 эпох, батч 30
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       157
           1       0.71      0.77      0.74       143

    accuracy                           0.74       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.74      0.74      0.74       300

